In [66]:
import time
import pandas as pd
import numpy as np

# 时间戳转字符串
def timestamp_datetime(value):
    format = '%Y-%m-%d %H:%M:%S'
    value = time.localtime(value)
    dt = time.strftime(format, value)
    return dt

### 提取特征

In [75]:
numeric_feats = ['item_price_level', 'item_sales_level', 'item_collected_level', 'item_pv_level', 'user_age_level',
                 'user_star_level', 'shop_review_num_level',
                 'shop_review_positive_rate', 'shop_star_level', 'shop_score_service', 'shop_score_delivery', 'shop_score_description',
                 'hour', 'day',
                 'user_query_day', 'user_query_day_hour',
                 'item_CTR', 'shop_CTR', 
                 ]

nominal_feats = ['item_id', 'user_id', 'shop_id', 'item_brand_id',
                 'item_city_id', 'user_gender_id', 'user_occupation_id', 'context_page_id']

features = numeric_feats + nominal_feats

target = 'is_trade'

def extract_date(data):
    data['time'] = data.context_timestamp.apply(timestamp_datetime)
    data['day'] = data.time.apply(lambda x: int(x[8:10]))
    data['hour'] = data.time.apply(lambda x: int(x[11:13]))
    del data['time']
    return data

def extract_ctr(data, feature, alias):
    '''统计给定数据的转化率
    
    '''
    query_cnt = data.groupby([feature]).size().reset_index().rename(columns={0: 'query_cnt'})
    conversion_cnt = data[data['is_trade'] == 1].groupby([feature]).size().reset_index().rename(columns={0: 'conversion_cnt'})
    ctr = pd.merge(query_cnt, conversion_cnt, how='left', on=[feature])
    ctr[['conversion_cnt']] = ctr[['conversion_cnt']].fillna(0)
    ctr[alias] = ctr['conversion_cnt'] / ctr['query_cnt']
    return ctr

def extract_history_ctr(data, feature, alias):
    '''统计每一天之前的历史转化率
    
    '''
    last_data = None
    for day in range(18, 26):
        # 每一天之前的历史数据
        history_data = data[data['day'] == day-1]
        ctr = extract_ctr(history_data, feature, alias)     
        ctr['day'] = day 
        
        now_data = data[data['day'] == day]           
        now_data = pd.merge(now_data, ctr, how='left', on=[feature, 'day'])
        
              
        
        
        
        
        if not last_data is None :
            now_data = pd.concat([last_data, now_data])
        
        
        
        if day == 18:
            mean = 0
            std = 0
            alpha = 0
            beta = 0

        else:         
            mean = float(now_data[alias].mean())
            std = float(now_data[alias].std())
            alpha = mean * (mean * (1 - mean) / (std * std) - 1)
            beta = (1 - mean) * (mean * (1 - mean) / (std * std) - 1)
        
        now_data[alias] = (now_data['conversion_cnt'] + alpha) / (now_data['query_cnt'] + beta + alpha)       
        now_data[[alias]] = now_data[[alias]].fillna(mean)      
        last_data = now_data
        del now_data['query_cnt']
        del now_data['conversion_cnt']
              
    return now_data
    

    

def extract_manual_features(data):
    
    user_query_day = data.groupby(['user_id', 'day']).size(
    ).reset_index().rename(columns={0: 'user_query_day'})
    data = pd.merge(data, user_query_day, how='left',
                    on=['user_id', 'day'])

    user_query_day_hour = data.groupby(['user_id', 'day', 'hour']).size(
    ).reset_index().rename(columns={0: 'user_query_day_hour'})
    data = pd.merge(data, user_query_day_hour, how='left',
                    on=['user_id', 'day', 'hour'])
    data_a = extract_history_ctr(data, 'shop_id', 'shop_CTR')
    data_b = extract_history_ctr(data_a, 'item_id', 'item_CTR')
    
    
    
    feature_with_CTR = ['shop_id', 'item_id', 'day']
    data_b[feature_with_CTR] = data_b[feature_with_CTR].astype(np.int64)
    
#     data = pd.get_dummies(data, dummy_na=True, columns=['user_gender_id', 'user_occupation_id', 'context_page_id'])

    
#     data = data.replace(to_replace=[-1], value=np.NaN)
#     data[numeric_feats] = data[numeric_feats].fillna(data.mean())
#     data[numeric_feats] = data[numeric_feats].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

    return data_b

### 读取训练样本并转换

In [76]:
df_train = pd.read_csv("../round1_ijcai_18_train_20180301.txt", sep=' ')
df_test = pd.read_csv("../round1_ijcai_18_test_a_20180301.txt", sep=' ')

df_train.drop_duplicates(inplace=True)
df_train = extract_date(df_train)
df_test = extract_date(df_test)

num_train = df_train.shape[0]
all_data = pd.concat([df_train, df_test])
all_data = extract_manual_features(all_data)
df_train = all_data[:num_train]
df_test = all_data[num_train:]
del df_test['is_trade']


# 根据日期，取24号为验证集
train_data = df_train.loc[df_train.day < df_train['day'].max()]
val_data = df_train.loc[df_train.day == df_train['day'].max()]

train_data.shape, val_data.shape

((420693, 33), (57418, 33))

In [78]:
df_train.head

<bound method NDFrame.head of                 context_id  context_page_id  context_timestamp  day  hour  \
0       282924576738839389             4006         1537236544   18    10   
1      4007979028023783431             4001         1537243232   18    12   
2      4884875192608989870             4001         1537211052   18     3   
3       840119421106178602             4016         1537222670   18     6   
4      1736769971710354684             4001         1537271320   18    19   
5      4434980272230296456             4003         1537282855   18    23   
6      3622211816051289512             4001         1537280317   18    22   
7      7851031132945961016             4001         1537261120   18    16   
8      8388974876851097582             4001         1537208871   18     2   
9      1138535512266486347             4003         1537285390   18    23   
10     1321094267098227649             4001         1537263036   18    17   
11     3293182224165193298             4001   

In [79]:
extract_ctr(df_train, feature='user_gender_id', alias='page_CTR')

,user_gender_id,query_cnt,conversion_cnt,page_CTR
0,-1,12902,166,0.012866
1,0,360797,6646,0.018420
2,1,94063,1987,0.021124
3,2,10349,195,0.018842


In [80]:
numeric_feats = ['item_price_level', 'item_sales_level', 'item_collected_level', 'item_pv_level', 'user_age_level',
                 'user_star_level', 'shop_review_num_level',
                 'shop_review_positive_rate', 'shop_star_level', 'shop_score_service', 'shop_score_delivery', 'shop_score_description',
                 'hour', 'day',
                 'user_query_day', 'user_query_day_hour',
                 'item_CTR', 'shop_CTR', 
                 ]

nominal_feats = ['item_id', 'user_id', 'shop_id', 'item_brand_id',
                 'item_city_id', 'user_gender_id', 'user_occupation_id', 'context_page_id']

features = numeric_feats + nominal_feats

target = 'is_trade'

train_data.columns

Index(['context_id', 'context_page_id', 'context_timestamp', 'day', 'hour',
       'instance_id', 'is_trade', 'item_CTR', 'item_brand_id',
       'item_category_list', 'item_city_id', 'item_collected_level', 'item_id',
       'item_price_level', 'item_property_list', 'item_pv_level',
       'item_sales_level', 'predict_category_property', 'shop_CTR', 'shop_id',
       'shop_review_num_level', 'shop_review_positive_rate',
       'shop_score_delivery', 'shop_score_description', 'shop_score_service',
       'shop_star_level', 'user_age_level', 'user_gender_id', 'user_id',
       'user_occupation_id', 'user_query_day', 'user_query_day_hour',
       'user_star_level'],
      dtype='object')

### lightgbm

In [81]:
from sklearn.metrics import make_scorer, precision_score, recall_score, accuracy_score, log_loss
import lightgbm as lgb
clf = lgb.LGBMClassifier(max_depth=4, n_estimators=100, n_jobs=-1)

clf.fit(train_data[features], train_data[target], feature_name=features, categorical_feature=['user_gender_id'])

loss_train = log_loss(train_data[target], clf.predict_proba(train_data[features]))
loss_val = log_loss(val_data[target], clf.predict_proba(val_data[features]))

loss_train, loss_val

D:\Program Files (x86)\Anaconda3\lib\site-packages\lightgbm\basic.py:1038: UserWarning: categorical_feature in Dataset is overrided. New categorical_feature is ['user_gender_id']
  warnings.warn('categorical_feature in Dataset is overrided. New categorical_feature is {}'.format(sorted(list(categorical_feature))))


(0.0881588029594573, 0.08184904310767672)

### 训练和测试

In [294]:
from sklearn.metrics import make_scorer, precision_score, recall_score, accuracy_score, log_loss
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(penalty="l2", solver='liblinear',
                        max_iter=1000, verbose=1)

from sklearn.ensemble import GradientBoostingClassifier
gbdt = GradientBoostingClassifier(n_estimators=100, max_features='auto')

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, n_jobs=-1, max_depth=5, bootstrap=True, max_features='auto')

clf = gbdt
clf.fit(train_data[features], train_data[target])

loss_train = log_loss(train_data[target], clf.predict_proba(train_data[features]))
predicted = clf.predict_proba(val_data[features])
loss_val = log_loss(val_data[target], predicted)

loss_train, loss_val

(0.088690767298507736, 0.081663202197175475)

# 提交测试数据

In [325]:


clf = lgb.LGBMClassifier(max_depth=4, n_estimators=100, n_jobs=-1)
clf.fit(df_train[features], df_train[target], feature_name=features, categorical_feature=['user_gender_id'])

loss_train = log_loss(df_train[target], clf.predict_proba(df_train[features]))

df_test['predicted_score'] = clf.predict_proba(df_test[features])[:, 1]

df_test[['instance_id', 'predicted_score']].to_csv(
    '20180325.txt', index=False, sep=' ')

loss_train

/home/user02/miniconda3/envs/gluon/lib/python3.6/site-packages/lightgbm/basic.py:1038: UserWarning: categorical_feature in Dataset is overrided. New categorical_feature is ['user_gender_id']
  warnings.warn('categorical_feature in Dataset is overrided. New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/home/user02/miniconda3/envs/gluon/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


0.087222402067231086